# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686546


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:47,  1.68s/it]

Rendering models:  10%|█         | 3/30 [00:03<00:35,  1.33s/it]

Rendering models:  13%|█▎        | 4/30 [00:04<00:29,  1.12s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:24,  1.03it/s]

...

Rendering models:  87%|████████▋ | 26/30 [00:18<00:01,  2.48it/s]

Rendering models:  90%|█████████ | 27/30 [00:18<00:01,  2.47it/s]

Rendering models:  93%|█████████▎| 28/30 [00:19<00:00,  2.30it/s]

Rendering models:  97%|█████████▋| 29/30 [00:19<00:00,  2.69it/s]

hailstorm3812                         0.000286
not-logged-in-0ed3d164fd32cee0b7f7    0.000158
not-logged-in-bc5fdb9ea8d611e60b8b    0.000091
richiemac                             0.000269
Dynamatt                              0.000121
not-logged-in-619dfe56b3aafaf4e45d    0.107528
not-logged-in-f7e2260ed4a9517a3e4a    1.432885
austingasbarra                        0.000129
shocko61                              0.000188
Enfysh                                0.000249
not-logged-in-6c3c85931d83a1164a0d    0.001060
CScampstudent                         0.592114
sn346808                              0.043385
LeeJoseph                             0.010859
caseyroberto                          0.000271
manik17                               0.000108
VladyslavPsyuk                        0.000944
not-logged-in-be875f27f6119e1941de    0.001064
not-logged-in-7be2ab5b99cd33eba4f5    0.011658
not-logged-in-7be2ab5b99cd33eba4f5    0.150906
SilliconStar                          0.000609
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())